In [ ]:
import os
print(os.getcwd())

In [ ]:
# 필요한 라이브러리 및 패키지 설치
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install -q datasets
!pip install nltk

In [ ]:
from torchvision.datasets import CocoCaptions
from torch.utils.data import Dataset, DataLoader

# COCO 데이터셋 다운로드 및 로드
coco_dataset = CocoCaptions(root="content/drive/MyDrive/abnormal_dataset", annFile="content/drive/MyDrive/abnormal_dataset.json", transform=None)
test_dataset = CocoCaptions(root="content/drive/MyDrive/test_abnormal_dataset", annFile="content/drive/MyDrive/test_abnormal_dataset/test_abnormal_dataset.json", transform=None)

# Hugging Face의 transformers 라이브러리에서 AutoProcessor 및 BlipForConditionalGeneration 불러오기
from transformers import AutoProcessor, BlipForConditionalGeneration

# Processor 및 모델 초기화
train_processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
valid_processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
test_processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
# 이미지 캡션 데이터셋 클래스 정의
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        # 데이터셋과 프로세서를 초기화
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        # 데이터셋의 길이
        return len(self.dataset)

    def __getitem__(self, idx):
        image, captions = self.dataset[idx]
        text = captions[0]

        encoding = self.processor(images=image, text=text, padding="max_length", return_tensors="pt")
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        return encoding

In [ ]:
from torch.utils.data import random_split

# 학습 데이터셋 크기를 전체의 90%로 설정
total_size = len(coco_dataset)
train_size = int(0.9 * total_size)
valid_size = (total_size - train_size)

# random_split을 사용하여 학습과 검증 데이터셋으로 분할
train_dataset, valid_dataset = random_split(coco_dataset, [train_size, valid_size])

# 학습, 검증, 테스트 데이터셋 초기화
train_dataset = ImageCaptioningDataset(train_dataset, train_processor)
valid_dataset = ImageCaptioningDataset(valid_dataset, valid_processor)
test_dataset = ImageCaptioningDataset(test_dataset, test_processor)

# 학습, 검증, 테스트용 데이터 로더 초기화
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)
valid_dataloader = DataLoader(valid_dataset, shuffle=False, batch_size=4)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1)

# 각 데이터셋의 샘플 수를 계산
train_size = len(train_dataset)
valid_size = len(valid_dataset)
test_size = len(test_dataset)

print(total_size)
print(f"Number of samples in the training dataset: {train_size}")
print(f"Number of samples in the validation dataset: {valid_size}")
print(f"Number of samples in the test dataset: {test_size}")